## Imports

In [1]:
import pandas as pd
from itertools import combinations,permutations
from pyvis.network import Network
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities, k_clique_communities, girvan_newman

In [2]:
df = pd.read_csv("../lyrics/songs_with_filenames_and_feats_and_lang_6.csv", delimiter=";")

In [3]:
attrs_df = pd.read_csv("artistinfo_extended2.csv", delimiter=";")

## Graph

In [4]:
def unpack_artist_list(str_repr):
    output = []
    try:
        output.extend(eval(str_repr))
    except:
        output.append(str_repr)
    finally:
        return output

In [5]:
edges = []
allartists = []

In [6]:
for song, artists in df[["song_name", "artists_names"]].itertuples(index=False):
    unpackartists = unpack_artist_list(artists)
    allartists.extend(unpackartists)
    pairs = list(combinations(unpackartists,2))
    edges.extend(pairs)
allartists = list(set(allartists))

In [7]:
edgedic = {}
for key in edges:
    if key in edgedic:
        edgedic[key] += 0.1
    else:
        edgedic[key] = 0.1
edges = list(edgedic.items())

In [8]:
G = nx.Graph()

In [9]:
G.add_nodes_from(allartists)

In [10]:
relevant = attrs_df[["artists_names", 'country', 'genres', 'type', 'tags', 'gender','end_area','name', 'life_end', 'life_ended', 'life_begin','begin_area', 'area']].fillna('')

In [11]:
dicts = relevant.groupby('artists_names').apply(lambda relevant: relevant.to_dict(orient='list')).to_dict()
nx.set_node_attributes(G, dicts)
for edge, count in edges:
    G.add_edge(edge[0],edge[1], weight=count)

In [12]:
#nx.write_gml(G, "artisti_coll.gml")
#H = nx.read_gml("artisti_coll.gml")

In [13]:
communities = k_clique_communities(G,4)

In [14]:
comms = []

In [15]:
for i in communities:
    if len(i) > 5:
        comms.extend(list(i))
        print (len(i))

136
16
15
24
20
7
6
6
6
8
7
6
6
8
6
8
6
6


In [16]:
largest_cc = max(nx.connected_components(G), key=len)

In [17]:
H = G.subgraph(comms)

In [18]:
g_degree = H.degree

In [19]:
options = '''
var options = {
  "nodes": {
    "color": {
      "border": "rgba(51, 204,0,1)",
      "background": "rgba(153, 255, 204,1)",
      "highlight": {
        "border": "rgba(51,233,148,1)",
        "background": "rgba(213,255,242,1)"
      },
      "hover": {
        "border": "rgba(59,233,157,1)",
        "background": "rgba(213,255,243,1)"
      }
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "hierarchicalRepulsion": {
      "centralGravity": 0
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  }
}
'''

In [20]:
net = Network(height='720px', width='1280px', bgcolor='#2e2f2f', font_color='EADEDA')
#net.show_buttons()
net.from_nx(H)
net.set_options(options)

In [21]:
for edge in net.edges:
    edge["value"] = edge["weight"]

In [22]:
for node in net.nodes:
    degree= g_degree[node['id']]
    node["degree"] = degree
    node['size'] = degree*0.5
    genres = node["genres"][0][1:-1].replace("'", "")
    life_end = node['life_end'][0][:4]
    area = ""
    if not life_end:
        life_end = "alive"
    if node['area'][0]:
        area = ' (' + node['area'][0]+')'
    quality = node['type'][0]
    if node['gender'][0]:
        quality = node['gender'][0]
    birth = node['life_begin'][0][:4]
    if not node['life_begin'][0]:
        birth = "?"
        
    tooltip = f"{node['artists_names'][0]}{area}<br>{quality}<br>{birth}-{life_end}{'<br>'+genres}"
    node['title'] = tooltip

In [23]:
net.show('nx.html')

In [24]:
#comunita rispetto a un attributo esterp
#assortativita rispetto a label